In [9]:
!pip install python-dotenv pydantic langchain langchain-groq langgraph --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.7 MB/s eta 0:00:00


In [5]:
import os, getpass
from dotenv import load_dotenv
load_dotenv(".env", override=True)

False

In [7]:
import os
import json
import re
from getpass import getpass
from google.colab import userdata
from dotenv import load_dotenv

load_dotenv(".env", override=True)

if "GROQ_API_KEY" not in os.environ:
    try:
        os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
    except Exception:
        os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API Key: ")


Enter your GROQ API Key: ··········


In [11]:
from pydantic import BaseModel, Field

class WeatherData(BaseModel):
    """Represents weather information."""

    temperature: float = Field(description="Temperature in Fahrenheit")
    wind_direction: str = Field(description="Wind direction (abbreviated)")
    wind_speed: float = Field(description="Wind speed in km/h")

In [16]:
from typing import Literal
from pydantic import BaseModel, Field  # Import for data validation and type hinting
from langchain_core.tools import tool  # Import for creating LangChain tools
from langchain_groq import ChatGroq  # Import for using the Groq language model
from langgraph.graph import MessagesState  # Import for managing chat messages in a graph

# Define a Pydantic model to represent weather data
class WeatherResponse(BaseModel):
    """Represents weather information."""
    temperature: float = Field(description="Temperature in Fahrenheit")
    wind_direction: str = Field(description="Wind direction (abbreviated)")
    wind_speed: float = Field(description="Wind speed in km/h")

# Define a class to represent the agent's state in the conversation
class AgentState(MessagesState):
    """
    Stores the state of the agent in the conversation.
    Inherits the 'messages' attribute from MessagesState to hold the chat history.
    """
    # Attribute to store the final structured response from the agent
    final_response: WeatherResponse

# Define a tool function to retrieve weather information
@tool
def get_weather(city: Literal['nyc', 'sf']) -> str:
    """
    Retrieves weather information for a specified city.
    Args:
        city: The city to get weather information for (either 'nyc' or 'sf').
    Returns:
        A string containing the weather information for the specified city.
    """
    if city == 'nyc':
        return 'It is cloudy in NYC, with 5 mph winds in the North-East direction and a temperature of 70 degrees'
    elif city == 'sf':
        return "It is 75 degrees and sunny in SF, with 3 mph winds in the South-East direction"  # Corrected "ph" to "mph"
    else:
        raise AssertionError("Unknown city")

# Create a list of tools to be used by the agent
tools = [get_weather]

# Initialize the Groq language model
model = ChatGroq(model='llama3-8b-8192')

# Bind the tools to the model
model_with_tools = model.bind_tools(tools)

# Configure the model to produce structured output using the WeatherData model
model_with_structured_output = model.with_structured_output(WeatherResponse)

## Option 1 single LLM option

### Define Graph

In [17]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode

tools = [get_weather, WeatherResponse]

#Force the model to use tools by passing tool_choice = 'any'

model_with_response_tool = model.bind_tools(tools,
                                            tool_choice='any')

# Define the function that calls the model
def call_model(state: AgentState):
    response = model_with_response_tool.invoke(state['messages'])

    # We return a list, because this will get added to the existing list
    return {'messages': [response]}

# Define the function that reponds to user
def respond(state: AgentState):
    # Construct the final answer from the arguments of the last tool call
    weather_tool_call = state['messages'][-1].tool_calls[0]
    response = WeatherResponse(**weather_tool_call['args'])

    #since we are using tool calling to return structured output
    # we need to add a tool message corresponding to the WeatherResponse tool call,
    # This is due to LLM providers' requirement that AI messages with tool calls
    # need to be follwed by a tool message for each tool call

    tool_message = {
        'type' : 'tool',
        'content': 'Here is your structured response',
        'tool_call_id' : weather_tool_call['id'],
    }

    # We return the final answer
    return {'final_response': response,
            'messages' : [tool_message]}

def should_continue(state: AgentState):
    message = state['messages']
    last_message = message[-1]
    # If there is only tool call and it is the response tool call we respond to the user

    if (
        len(last_message.tool_calls) == 1
        and
        last_message.tool_calls[0]['name'] == 'WeatherResponse'
    ):
        return 'respond'
    # Otherwise we will use the tool node again
    else:
        return 'continue'

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("respond",respond)
workflow.add_node('tools', ToolNode(tools))

# Set the entrypoint as 'agent'
# This means that this node is the first one called
workflow.set_entry_point('agent')

# We now add a conditional edge
workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'tools',
        'respond' : 'respond'
    }
)

workflow.add_edge('tools','agent')
workflow.add_edge('respond',END)
graph = workflow.compile()

## Usage

In [18]:
answer = graph.invoke(input={
                            'messages': [('human',
                                        "what's the weather in SF")]
                            })['final_response']

In [19]:
answer

WeatherResponse(temperature=75.0, wind_direction='S-E', wind_speed=3.0)

## Option 2: 2 LLMs

### Define Graph

In [20]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage


# Define the function that calls the model
def call_model(state: AgentState):
    response = model_with_tools.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the function that responds to the user
def respond(state: AgentState):
    # We call the model with structured output in order to return the same format to the user every time
    # state['messages'][-2] is the last ToolMessage in the convo, which we convert to a HumanMessage for the model to use
    # We could also pass the entire chat history, but this saves tokens since all we care to structure is the output of the tool
    response = model_with_structured_output.invoke(
        [HumanMessage(content=state["messages"][-2].content)]
    )
    # We return the final answer
    return {"final_response": response}


# Define the function that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we respond to the user
    if not last_message.tool_calls:
        return "respond"
    # Otherwise if there is, we continue
    else:
        return "continue"


# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("respond", respond)
workflow.add_node("tools", ToolNode(tools))

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "respond": "respond",
    },
)

workflow.add_edge("tools", "agent")
workflow.add_edge("respond", END)
graph = workflow.compile()

In [21]:
answer = graph.invoke(input={"messages": [("human", "what's the weather in SF?")]})[
    "final_response"
]

In [22]:
answer

WeatherResponse(temperature=75.0, wind_direction='South-East', wind_speed=3.0)